# For running the final deployment run the last cell only

# Importing Libraries

In [10]:
import numpy as np
import pandas as pd 
import string
import warnings
warnings.filterwarnings('ignore')
import re

#from textblob import TextBlob

import nltk
#nltk.download('punkt')
#nltk.download('wordnet')

import spacy

from nltk.corpus import stopwords

#Stemming
#from nltk.stem import PorterStemmer
#stemmer = PorterStemmer()

#Lemmetization
from nltk.stem import WordNetLemmatizer
lemmetizer = WordNetLemmatizer()

from flask import Flask, redirect, url_for, request,render_template

#Initializing spacy model
nlp = spacy.load('en_core_web_sm')

#Words that we do not require after tokenization in the search term and the title column as well
unwanted_words = ['Email','Sample','Letter','Template','email','sample','letter','template','to','To']

#Punctuations
#punctuations = '''()-[]{};:'"\,<>./?@#$%^&*_`~'''

#Stopwords
stop_words = stopwords.words('english')

# Importing Dataset and Data Preprocessing

In [11]:
#Reading the data set and preprocessing
data = pd.read_excel('templates.xls')
data = data.dropna()
data = data.reset_index(drop=True)

#Replacing 'Letter' with 'Email' || Replacing '-' with ' ' in the cat column
for i in range (0,len(data)):
    data.iloc[i,0] = data.iloc[i,0].replace('Letter','Email')
    data.iloc[i,0] = data.iloc[i,0].replace('letter','Email')
    data.iloc[i,1] = data.iloc[i,1].replace('Letter','Email')
    data.iloc[i,1] = data.iloc[i,1].replace('letter','Email')
    data.iloc[i,0] = data.iloc[i,0].replace('-',' ')

#Adding Search column which will contain tokens of title and arranging the columns in order    
data['Search_column'] = ''
data = data[["Category","Search_column", "Title", "Body"]]

#Removing the 'Download Related Samples...' from footer
for i in range (0,len(data)):
    txt = data.iloc[i,3]
    txt0 = txt.split('\nDownload ')[0]
    data.iloc[i,3] = txt0

#Removing address fields
for i in range (0,len(data)):
    list0 = data.Body[i].split('\n')
    list1 = []
    for string in list0:
        if len(string)> 50:
            list1.append(string)
    body = 'To: [Email id]\n\nSubject: [Subject]\n\n[Salutation]\n\n'
    for j in list1:
        body = body + j + '\n\n'
    body = body + '[Closure]\n\n'+'[Name/Signature]\n'+'[Designation]\n'+'[Organization]\n'
    body = body.replace('Letter','Email')
    body = body.replace('letter','email')
    #print(body)
    data.iloc[i,3] = body

#Preprocessing the title column
for i in range (0,len(data)):
    tex = str(data.Title[i])
    tex_p = ''
    #for char in tex:
    #    if (char not in punctuations):
    #        tex_p = tex_p+char
    tex_p = re.sub(r'[^\w\s]','',tex)
    tex_p = tex_p.strip()
    tex_a = nltk.word_tokenize(tex_p)
    tex_s=[]
    for word in tex_a:
        if (word not in stop_words):
            tex_s.append(word)
    tex_l = []
    for word in tex_s:
        tex_l.append(lemmetizer.lemmatize(word))
    tex_f = []
    for word in tex_l:
        if word not in unwanted_words:
            tex_f.append(word)
        else:
            pass
    data.iloc[i,1] = tex_f

#Named Entity Recognition (Replacing named entities with their tags)
for i in range(0,len(data)):
    s= data.Body[i]
    doc = nlp(s)
    newString = s
    for e in reversed(doc.ents): #reversed to not modify the offsets of other entities when substituting
        start = e.start_char
        end = start + len(e.text)
        newString = newString[:start] + '[' + e.label_ + ']' + newString[end:]
    data.Body[i] = newString
    
data.head()

,Category,Search_column,Title,Body
0,Acceptance Emails,"[Elementary, School, Admission]",Elementary School Admission Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
1,Acceptance Emails,"[Academy, Admission]",Academy Admission Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
2,Acceptance Emails,"[College, Admissions, Recommendation]",College Admissions Recommendation Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
3,Acceptance Emails,"[Decline, Admission]",Decline Admission Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
4,Acceptance Emails,"[Harvard, Acceptance]",Harvard Acceptance Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...


# EDA

In [4]:
data.shape

(1574, 4)

Dataset contains 1574 entries and 4 columns

In [5]:
data.columns

Index(['Category', 'Search_column', 'Title', 'Body'], dtype='object')

The dataset consists of 4 columns namely 'Category', 'Search_column', 'Title' and 'Body'.
<br>Dataset altogether consists of 1574 Email templates.

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1574 entries, 0 to 1573
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Category       1574 non-null   object
 1   Search_column  1574 non-null   object
 2   Title          1574 non-null   object
 3   Body           1574 non-null   object
dtypes: object(4)
memory usage: 49.3+ KB


As we can see that there are no null values.

In [7]:
print('Length: '+str(len(data.Category.unique()))+'\n')
data.Category.unique()

Length: 53



array(['Acceptance Emails', 'Admission Emails', 'Announcement Emails',
       'Application Emails', 'Appreciation Emails',
       'Authorization Emails', 'Banking Emails', 'Birthday Email',
       'Business Emails', 'Cancellation Email', 'Certification Email',
       'Claim Email', 'Complaint Emails', 'Condolence Emails',
       'Confirmation Email', 'Congratulation Emails', 'Consumer Emails',
       'Customer Emails', 'Decline Email', 'Emails', 'Employment Emails',
       'Finance Emails', 'Formal Email', 'General Emails',
       'Government Emails', 'Grievance Emails', 'Holiday Emails',
       'Inquiry Email', 'Internship Email', 'Interview Email',
       'Introduction Emails', 'Invitation Emails', 'Job Emails',
       'Leave Emails', 'Marketing Emails', 'Order Email',
       'Permission Email', 'Personal Emails', 'Recommendation Emails',
       'Reference Emails', 'Request Emails', 'Resignation Emails',
       'Retirement Emails', 'Sales & Marketing Emails', 'Sales Emails',
       '

As we can see that there are 53 Categories as shown above.

The title column shows the title of each Email

The Search_Column consists of the tokens from the title column for matching the keywords.

The Body column consists of templates of the email.

# Search Term Input and Processing

In [3]:
g_text = input('Search Email Templates:\n\n')

Search Email Templates:

Sick Leave email template to Boss?"?"


In [4]:
text=''
text = re.sub(r'[^\w\s]','',g_text)
text

'Sick Leave email template to Boss'

In [5]:
#text=''
#for char in g_text:
#    if (char not in punctuations):
#        text = text+char
#text

In [6]:
text = text.strip()
text = text.title()
text

'Sick Leave Email Template To Boss'

In [7]:
text_a = nltk.word_tokenize(text)
text_a

['Sick', 'Leave', 'Email', 'Template', 'To', 'Boss']

In [8]:
text_s=[]
for word in text_a:
    if (word not in stop_words):
        text_s.append(word)
        
text_s

['Sick', 'Leave', 'Email', 'Template', 'To', 'Boss']

In [13]:
text_l = []
for word in text_s:
    if word == 'boss':
        text_l.append(word)
    else: 
        text_l.append(lemmetizer.lemmatize(word))
text_l

['Sick', 'Leave', 'Email', 'Template', 'To', 'Boss']

In [14]:
text_f = []
for word in text_l:
    if word not in unwanted_words:
        text_f.append(word)
    else:
        pass 
text_f

['Sick', 'Leave', 'Boss']

# Searching for keywords in Search_column

In [15]:
data.head()

,Category,Search_column,Title,Body
0,Acceptance Emails,"[Elementary, School, Admission]",Elementary School Admission Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
1,Acceptance Emails,"[Academy, Admission]",Academy Admission Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
2,Acceptance Emails,"[College, Admissions, Recommendation]",College Admissions Recommendation Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
3,Acceptance Emails,"[Decline, Admission]",Decline Admission Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
4,Acceptance Emails,"[Harvard, Acceptance]",Harvard Acceptance Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...


In [16]:
#Searching for keywords in the search column
result_in = []
for word in text_f:
    for i in range (0,len(data)):
        for j in range (0,len(data.iloc[i,1])):
            if data.iloc[i,1][j] not in text_f:
                pass
            else:
                result_in.append(int(i))
result_in.sort(reverse=False)
result_in

[129,
 129,
 129,
 162,
 162,
 162,
 257,
 257,
 257,
 431,
 431,
 431,
 463,
 463,
 463,
 978,
 978,
 978,
 979,
 979,
 979,
 980,
 980,
 980,
 981,
 981,
 981,
 982,
 982,
 982,
 983,
 983,
 983,
 984,
 984,
 984,
 985,
 985,
 985,
 986,
 986,
 986,
 987,
 987,
 987,
 987,
 987,
 987,
 989,
 989,
 989,
 990,
 990,
 990,
 991,
 991,
 991,
 995,
 995,
 995,
 997,
 997,
 997,
 998,
 998,
 998,
 999,
 999,
 999,
 1000,
 1000,
 1000,
 1001,
 1001,
 1001,
 1003,
 1003,
 1003,
 1004,
 1004,
 1004,
 1203,
 1203,
 1203,
 1225,
 1225,
 1225,
 1265,
 1265,
 1265,
 1486,
 1486,
 1486,
 1493,
 1493,
 1493]

In [17]:
x = data.iloc[result_in]
x['priority'] = np.NaN
x.head()

,Category,Search_column,Title,Body,priority
129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN
129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN
129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN
162,Appreciation Emails,"[Appreciation, Boss]",Appreciation Email to Boss,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN
162,Appreciation Emails,"[Appreciation, Boss]",Appreciation Email to Boss,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN


In [18]:
p = x.index.value_counts()
p

987     6
1493    3
984     3
162     3
257     3
431     3
1203    3
1225    3
1486    3
463     3
978     3
979     3
980     3
981     3
982     3
983     3
985     3
1265    3
986     3
989     3
990     3
991     3
995     3
997     3
998     3
999     3
1000    3
1001    3
1003    3
1004    3
129     3
dtype: int64

In [19]:
x = x.reset_index()
p = p.reset_index()
p = p.rename(columns={0:'priority'},inplace=False)

In [20]:
x.head()

,index,Category,Search_column,Title,Body,priority
0,129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN
1,129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN
2,129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN
3,162,Appreciation Emails,"[Appreciation, Boss]",Appreciation Email to Boss,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN
4,162,Appreciation Emails,"[Appreciation, Boss]",Appreciation Email to Boss,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN


In [21]:
p.head()

,index,priority
0,987,6
1,1493,3
2,984,3
3,162,3
4,257,3


In [22]:
for i in range(0,len(x)):
    ind = x['index'][i] 
    for j in range(0,len(p)):
        if p['index'][j] == ind:
            pr = p['priority'][j]
        else:
            pass
    x['priority'][i] = pr
x.head()

,index,Category,Search_column,Title,Body,priority
0,129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,3.0
1,129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,3.0
2,129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,3.0
3,162,Appreciation Emails,"[Appreciation, Boss]",Appreciation Email to Boss,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,3.0
4,162,Appreciation Emails,"[Appreciation, Boss]",Appreciation Email to Boss,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,3.0


In [23]:
x = x.sort_values(by=['priority'],ascending=False)
x.head()

,index,Category,Search_column,Title,Body,priority
47,987,Leave Emails,"[Leave, Boss]",Leave Email To Boss,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,6.0
46,987,Leave Emails,"[Leave, Boss]",Leave Email To Boss,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,6.0
45,987,Leave Emails,"[Leave, Boss]",Leave Email To Boss,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,6.0
44,987,Leave Emails,"[Leave, Boss]",Leave Email To Boss,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,6.0
43,987,Leave Emails,"[Leave, Boss]",Leave Email To Boss,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,6.0


# Results

In [24]:
x = x.sort_values(by=['priority'],ascending=False)
x = x.drop_duplicates(subset=['index'])
x = x.drop(['index','Search_column','priority'],axis=1)
x = x.reset_index(drop=True)
x.head()

,Category,Title,Body
0,Leave Emails,Leave Email To Boss,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
1,Congratulation Emails,Congratulation to Boss,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
2,Appreciation Emails,Appreciation Email to Boss,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
3,Business Emails,Short Request for Leave of Absence,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
4,Condolence Emails,Condolence Email to Boss,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...


In [25]:
print('Category: ',x.Category[0],'\n')
print('Title: ',x.Title[0])
print('____________________________________________________________________________________________________________________________\n')
print(x.Body[0])

Category:  Leave Emails 

Title:  Leave Email To Boss
____________________________________________________________________________________________________________________________

To: [Email id]

Subject: [Subject]

[Salutation]

As I have discussed with you in the past that I am having an issue with my family. My mother is ill and I will need to go home and take care of her. Therefore I am formally asking for a leave of absence, starting on [DATE].

I am hoping to be back at work on [DATE] .If there is any delay of any sort, I will let you know. If there is any way I can do any work remotely during that time, please let me know as I don’t want to neglect  my duties and still wish to help the company as much as I can. My  family and I are all hoping for my mother to have a speed and full  recovery, and we all greatly appreciate your support during this  difficult time.

[Closure]

[Name/Signature]
[Designation]
[Organization]



## Just for future reference

In [26]:
df = pd.DataFrame(data.Category.value_counts())
df = df.reset_index()
df = df.rename(columns={'index':'Category','Category':'N_Letters'})
df.head()

,Category,N_Letters
0,Personal Emails,70
1,Business Emails,70
2,Customer Emails,70
3,General Emails,64
4,Sales & Marketing Emails,60


# Deployment

In [9]:
app = Flask(__name__)

#app.static_folder = 'static'


@app.route('/')
def home_page():
    cats = df.Category.to_list()
    return render_template('index.html',cat_list = cats)



@app.route('/category',methods = ['POST','GET'])
def category():
    cats = df.Category.to_list()
    if request.method == 'POST':
        s = str(request.form['cat'])
        b = data[data.Category == s]
        title = b.Title.to_list()
    return render_template('cat_title.html',title_list=title,cat_list = cats)



@app.route('/category/title',methods = ['POST','GET'])
def cat_title():
    if request.method == 'POST':
        s = str(request.form['title'])
        b = data[data.Title == s]
        b = b.reset_index(drop=True)
        letter = 'Category: '+str(b.Category[0])+'\nTitle: '+str(b.Title[0])+'\n____________________________________________________________________________________________________________\n'+str(b.Body[0])
        letter = letter.split('\n')
        cats = df.Category.to_list()
        c = data[data.Category == b.Category[0]]
        title = c.Title.to_list()
    return render_template('print.html',title_list = title,cat_list = cats,letter=letter)


        
@app.route('/search_results',methods = ['POST','GET'])
def search():
    if request.method == 'POST':
        s = str(request.form['keywords'])
        text=''
        text = re.sub(r'[^\w\s]','',s)
        text = text.strip()
        text = text.title()
        text_a = nltk.word_tokenize(text)
        text_s=[]
        for word in text_a:
            if (word not in stop_words):
                text_s.append(word)
        text_l = []
        for word in text_s:
            if word == 'boss':
                text_l.append(word)
            else: 
                text_l.append(lemmetizer.lemmatize(word))
        text_f = []
        for word in text_l:
            if word not in unwanted_words:
                text_f.append(word)
            else:
                pass
        #Searching for keywords in the search column
        result_in = []
        for word in text_f:
            for i in range (0,len(data)):
                for j in range (0,len(data.iloc[i,1])):
                    if data.iloc[i,1][j] not in text_f:
                        pass
                    else:
                        result_in.append(int(i))
        result_in.sort(reverse=False)
        x = data.iloc[result_in]
        x['priority'] = np.NaN
        p = x.index.value_counts()
        x = x.reset_index()
        p = p.reset_index()
        p = p.rename(columns={0:'priority'},inplace=False)
        for i in range(0,len(x)):
            ind = x['index'][i] 
            for j in range(0,len(p)):
                if p['index'][j] == ind:
                    pr = p['priority'][j]
                else:
                    pass
            x['priority'][i] = pr
        x = x.sort_values(by=['priority'],ascending=False)
        x = x.drop_duplicates(subset=['index'])
        x = x.drop(['index','Search_column','priority'],axis=1)
        x = x.reset_index(drop=True)
        #letter = 'Category: '+str(x.Category[0])+'\nTitle: '+str(x.Title[0])+'\n____________________________________________________________________________________________________________\n'+str(x.Body[0])
        #letter = letter.split('\n')
        cats = df.Category.to_list()
        title_list = x.Title.to_list()
    return render_template('keywordsearch.html',cat_list = cats,title_list=title_list)



@app.route('/search_results/print',methods = ['POST','GET'])
def print_search_results():
    if request.method == 'POST':
        s = str(request.form['title'])
        b = data[data.Title == s]
        b = b.reset_index(drop=True)
        letter = 'Category: '+str(b.Category[0])+'\nTitle: '+str(b.Title[0])+'\n____________________________________________________________________________________________________________\n'+str(b.Body[0])
        letter = letter.split('\n')
        cats = df.Category.to_list()
        c = data[data.Category == b.Category[0]]
        title = c.Title.to_list()
    return render_template('searchresultprint.html',letter = letter,cat_list = cats,title_list=title)




if __name__ == '__main__':
    app.run() 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Oct/2021 18:37:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Oct/2021 18:37:12] "GET /static/titleicon.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Oct/2021 18:37:51] "POST /search_results HTTP/1.1" 200 -
127.0.0.1 - - [09/Oct/2021 18:38:03] "POST /search_results/print HTTP/1.1" 200 -
127.0.0.1 - - [09/Oct/2021 18:38:16] "POST /search_results/print HTTP/1.1" 200 -
127.0.0.1 - - [09/Oct/2021 18:38:51] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [09/Oct/2021 18:39:15] "POST /category/title HTTP/1.1" 200 -
127.0.0.1 - - [09/Oct/2021 18:39:27] "POST /category/title HTTP/1.1" 200 -
127.0.0.1 - - [09/Oct/2021 18:40:02] "POST /category/title HTTP/1.1" 200 -


# Final Combined Deployment

In [ ]:
#****Importing Libraries****#

import numpy as np
import pandas as pd 
import string
import warnings
warnings.filterwarnings('ignore')
import re

#from textblob import TextBlob

import nltk
#nltk.download('punkt')
#nltk.download('wordnet')

import spacy

from nltk.corpus import stopwords

#Stemming
#from nltk.stem import PorterStemmer
#stemmer = PorterStemmer()

#Lemmetization
from nltk.stem import WordNetLemmatizer
lemmetizer = WordNetLemmatizer()

from flask import Flask, redirect, url_for, request,render_template

#Initializing spacy model
nlp = spacy.load('en_core_web_sm')

#Words that we do not require after tokenization in the search term and the title column as well
unwanted_words = ['Email','Sample','Letter','Template','email','sample','letter','template','to','To']

#Punctuations
#punctuations = '''()-[]{};:'"\,<>./?@#$%^&*_`~'''

#Stopwords
stop_words = stopwords.words('english')


#****Importing Dataset and data Preprocessing****#

#Reading the data set and preprocessing
data = pd.read_excel('templates.xls')
data = data.dropna()
data = data.reset_index(drop=True)

#Replacing 'Letter' with 'Email' || Replacing '-' with ' ' in the cat column
for i in range (0,len(data)):
    data.iloc[i,0] = data.iloc[i,0].replace('Letter','Email')
    data.iloc[i,0] = data.iloc[i,0].replace('letter','Email')
    data.iloc[i,1] = data.iloc[i,1].replace('Letter','Email')
    data.iloc[i,1] = data.iloc[i,1].replace('letter','Email')
    data.iloc[i,0] = data.iloc[i,0].replace('-',' ')

#Adding Search column which will contain tokens of title and arranging the columns in order    
data['Search_column'] = ''
data = data[["Category","Search_column", "Title", "Body"]]

#Removing the 'Download Related Samples...' from footer
for i in range (0,len(data)):
    txt = data.iloc[i,3]
    txt0 = txt.split('\nDownload ')[0]
    data.iloc[i,3] = txt0

#Removing address fields
for i in range (0,len(data)):
    list0 = data.Body[i].split('\n')
    list1 = []
    for string in list0:
        if len(string)> 50:
            list1.append(string)
    body = 'To: [Email id]\n\nSubject: [Subject]\n\n[Salutation]\n\n'
    for j in list1:
        body = body + j + '\n\n'
    body = body + '[Closure]\n\n'+'[Name/Signature]\n'+'[Designation]\n'+'[Organization]\n'
    body = body.replace('Letter','Email')
    body = body.replace('letter','email')
    #print(body)
    data.iloc[i,3] = body

#Preprocessing the title column
for i in range (0,len(data)):
    tex = str(data.Title[i])
    tex_p = ''
    #for char in tex:
    #    if (char not in punctuations):
    #        tex_p = tex_p+char
    tex_p = re.sub(r'[^\w\s]','',tex)
    tex_p = tex_p.strip()
    tex_a = nltk.word_tokenize(tex_p)
    tex_s=[]
    for word in tex_a:
        if (word not in stop_words):
            tex_s.append(word)
    tex_l = []
    for word in tex_s:
        tex_l.append(lemmetizer.lemmatize(word))
    tex_f = []
    for word in tex_l:
        if word not in unwanted_words:
            tex_f.append(word)
        else:
            pass
    data.iloc[i,1] = tex_f

#Named Entity Recognition (Replacing named entities with their tags)
for i in range(0,len(data)):
    s= data.Body[i]
    doc = nlp(s)
    newString = s
    for e in reversed(doc.ents): #reversed to not modify the offsets of other entities when substituting
        start = e.start_char
        end = start + len(e.text)
        newString = newString[:start] + '[' + e.label_ + ']' + newString[end:]
    data.Body[i] = newString
    

#****For future reference****#

df = pd.DataFrame(data.Category.value_counts())
df = df.reset_index()
df = df.rename(columns={'index':'Category','Category':'N_Letters'})


#****Deployment****#

app = Flask(__name__)

#app.static_folder = 'static'


@app.route('/')
def home_page():
    cats = df.Category.to_list()
    return render_template('index.html',cat_list = cats)



@app.route('/category',methods = ['POST','GET'])
def category():
    cats = df.Category.to_list()
    if request.method == 'POST':
        s = str(request.form['cat'])
        b = data[data.Category == s]
        title = b.Title.to_list()
    return render_template('cat_title.html',title_list=title,cat_list = cats)



@app.route('/category/title',methods = ['POST','GET'])
def cat_title():
    if request.method == 'POST':
        s = str(request.form['title'])
        b = data[data.Title == s]
        b = b.reset_index(drop=True)
        letter = 'Category: '+str(b.Category[0])+'\nTitle: '+str(b.Title[0])+'\n____________________________________________________________________________________________________________\n'+str(b.Body[0])
        letter = letter.split('\n')
        cats = df.Category.to_list()
        c = data[data.Category == b.Category[0]]
        title = c.Title.to_list()
    return render_template('print.html',title_list = title,cat_list = cats,letter=letter)


        
@app.route('/search_results',methods = ['POST','GET'])
def search():
    if request.method == 'POST':
        s = str(request.form['keywords'])
        text=''
        text = re.sub(r'[^\w\s]','',s)
        text = text.strip()
        text = text.title()
        text_a = nltk.word_tokenize(text)
        text_s=[]
        for word in text_a:
            if (word not in stop_words):
                text_s.append(word)
        text_l = []
        for word in text_s:
            if word == 'boss':
                text_l.append(word)
            else: 
                text_l.append(lemmetizer.lemmatize(word))
        text_f = []
        for word in text_l:
            if word not in unwanted_words:
                text_f.append(word)
            else:
                pass
        #Searching for keywords in the search column
        result_in = []
        for word in text_f:
            for i in range (0,len(data)):
                for j in range (0,len(data.iloc[i,1])):
                    if data.iloc[i,1][j] not in text_f:
                        pass
                    else:
                        result_in.append(int(i))
        result_in.sort(reverse=False)
        x = data.iloc[result_in]
        x['priority'] = np.NaN
        p = x.index.value_counts()
        x = x.reset_index()
        p = p.reset_index()
        p = p.rename(columns={0:'priority'},inplace=False)
        for i in range(0,len(x)):
            ind = x['index'][i] 
            for j in range(0,len(p)):
                if p['index'][j] == ind:
                    pr = p['priority'][j]
                else:
                    pass
            x['priority'][i] = pr
        x = x.sort_values(by=['priority'],ascending=False)
        x = x.drop_duplicates(subset=['index'])
        x = x.drop(['index','Search_column','priority'],axis=1)
        x = x.reset_index(drop=True)
        #letter = 'Category: '+str(x.Category[0])+'\nTitle: '+str(x.Title[0])+'\n____________________________________________________________________________________________________________\n'+str(x.Body[0])
        #letter = letter.split('\n')
        cats = df.Category.to_list()
        title_list = x.Title.to_list()
    return render_template('keywordsearch.html',cat_list = cats,title_list=title_list)



@app.route('/search_results/print',methods = ['POST','GET'])
def print_search_results():
    if request.method == 'POST':
        s = str(request.form['title'])
        b = data[data.Title == s]
        b = b.reset_index(drop=True)
        letter = 'Category: '+str(b.Category[0])+'\nTitle: '+str(b.Title[0])+'\n____________________________________________________________________________________________________________\n'+str(b.Body[0])
        letter = letter.split('\n')
        cats = df.Category.to_list()
        c = data[data.Category == b.Category[0]]
        title = c.Title.to_list()
    return render_template('searchresultprint.html',letter = letter,cat_list = cats,title_list=title)




if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
